# 生命游戏

这是个模拟生命演化的游戏，在一个广阔的生存空间里，设定生命群落存活和繁衍的规则，个体和群落依据既相互竞争又相互依存的法则进行进化。

## 总的规则

- 过于拥挤的分布 - 资源争夺 - 死亡
- 过于孤立的分布 - 不具备种群优势 - 灭绝
- 适度的分布 - 繁衍

## 规则细节


- 少于2邻居，死
- 多于3邻居，死
- 正好3邻居，生

## 创作者的解释

- [跳到1分钟处观看规则](http://bazhou.blob.core.windows.net/learning/mpp/414_Does_John_Conway_hate_his_Game_of_Life-E8kUJL04ELA.mp4 )

- [对于这个想法的来源（冯.诺伊曼）可以看这个视频](http://bazhou.blob.core.windows.net/learning/mpp/665_Inventing_Game_of_Life_-_Numberphile-R9Plq-D1gEk.mp4 )

## 交互体验

这个交互体验的目的是获得感官印象

- [体验游戏的过程是没有交互的](https://bitstorm.org/gameoflife/)，

## Wikipedia解释

如果已经明白规则，可以忽略这部分文档。

- [English version long](http://bazhou.blob.core.windows.net/learning/mpp/game_of_life_en.pdf)
- [汉语版本（短）](http://bazhou.blob.core.windows.net/learning/mpp/game_of_life_cn.pdf)

# 挑战

- 用Python实现游戏规则
  - 在1000x1000共一百万个单元里模拟
  - 边界为0
  - 不需要画图，只需要完成进化矩阵的运算即可
- 尽量短的代码
- 尽量高效

# 可以跳过下面全部的参考直接解决问题

## 参考一

[github搜索一个解](https://github.com/domoritz/gameoflife-python)

## 参考二

[Peter Norvig 的 notebook](https://nbviewer.jupyter.org/url/norvig.com/ipython/Life.ipynb)

## 参考三

[这个编程问题的96种语言实现](https://rosettacode.org/wiki/Conway%27s_Game_of_Life)

## 参考四

下面的代码是启发解，虽然不是最优解，但是

- 给出了算法的基本结构
  - 计算邻居数
  - 根据法则进化
- 循环结构指示了计算规模

In [214]:
import random as random
import timeit

# 产生一个百万0，1数组，0代表空（死），1代表生
Z = [[random.choice([0,1]) for x in range(1000)] for y in range(1000)]

# 计算八个邻居数目
def neighbours(Z):
    s = len(Z), len(Z[0])
    # 一个初始全为0的邻居数量矩阵
    N = [[0,]*(s[0]) for i in range(s[1])]
    for x in range(1, s[0]-1):                                  # 注意边界
        for y in range(1, s[1]-1):
            N[x][y] = (Z[x-1][y-1] + Z[x][y-1] + Z[x+1][y-1] +  # 邻居数量=周围8个格子生命统计
                       Z[x-1][y]               + Z[x+1][y]   +
                       Z[x-1][y+1] + Z[x][y+1] + Z[x+1][y+1])
    return N

# 根据周围邻居总数应用规则
def evolve(Z):
    s = len(Z), len(Z[0])
    N = neighbours(Z)
    for x in range(1, s[0]-1):
        for y in range(1, s[1]-1):
            if Z[x][y] == 1 and (N[x][y] < 2 or N[x][y] > 3): # 灭亡规则
                Z[x][y] = 0
            elif Z[x][y] == 0 and N[x][y] == 3:               # 繁衍规则
                Z[x][y] = 1
    return Z

print(timeit.timeit(lambda: evolve(Z), number=3))             # 对百万人口作三代进化，统计运算效率

6.1976799729745835


## 尝试用Numpy解

In [226]:
import timeit
import numpy as np

Znp = np.random.randint(2, size=(1000,1000))

def np_solver(Z):
        return Z # 把实现填进来
    
print(timeit.timeit(lambda: np_solver(Znp), number=3))

3.1669915188103914e-06


## 提示一

- 使用索引[1:-1,1:-1]可以消去邻居数循环

In [1]:
l = [1, 2, 3, 4, 5]
l[1:-1]

[2, 3, 4]

## 提示二

- [argwhere可以帮助作规则判断](https://docs.scipy.org/doc/numpy/reference/generated/numpy.argwhere.html)

In [21]:
import timeit
import numpy as np

Znp = np.random.randint(2, size=(1000,1000), dtype=np.int8)
Znp[0,:] = Znp[-1,:] = Znp[:,0] = Znp[:,-1] = 0

def no_loop(Z):
    N = np.zeros(Z.shape, dtype=int)
    N[1:-1,1:-1] += (Z[ :-2, :-2] + Z[ :-2,1:-1] + Z[ :-2,2:] +
                     Z[1:-1, :-2]                + Z[1:-1,2:] +
                     Z[2:  , :-2] + Z[2:  ,1:-1] + Z[2:  ,2:])


    Z[np.argwhere( (Z==1) & (N < 2) )] = 0
    Z[np.argwhere( (Z==1) & (N > 3) )] = 0
    Z[np.argwhere( (Z==0) & (N==3) )] = 0
    
print(timeit.timeit(lambda: no_loop(Znp), number=3))

0.13714909995906055


## 提示三

-[ravel是view](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ravel.html)

-[参考曾佑轩分享的关于Numpy性能的文章](https://zhuanlan.zhihu.com/p/28626431)

-[numpy.take](https://docs.scipy.org/doc/numpy/reference/generated/numpy.take.html)

-[numpy.compress](https://docs.scipy.org/doc/numpy/reference/generated/numpy.compress.html)

In [7]:
import timeit
import numpy as np

# 数据类型加速
Znp = np.random.randint(2, size=(1000,1000), dtype=np.int8)

def no_loop_int_view(Z):
    N = np.zeros(Z.shape, dtype=np.int8)
    N[1:-1,1:-1] += (Z[ :-2, :-2] + Z[ :-2,1:-1] + Z[ :-2,2:] +
                     Z[1:-1, :-2]                + Z[1:-1,2:] +
                     Z[2:  , :-2] + Z[2:  ,1:-1] + Z[2:  ,2:])

    N_ = N.ravel()
    Z_ = Z.ravel()

    R1 = np.argwhere((Z_== 1) & (N_ < 2))
    R2 = np.argwhere((Z_== 1) & (N_ > 3))
    R3 = np.argwhere((Z_== 1) & ((N_==2) | (N_==3)))
    R4 = np.argwhere((Z_== 0) & (N_==3))

    Z_[R1] = 0
    Z_[R2] = 0
    Z_[R3] = Z_[R3]
    Z_[R4] = 1

    Z[0,:] = Z[-1,:] = Z[:,0] = Z[:,-1] = 0
    
print(timeit.timeit(lambda: no_loop_int_view(Znp), number=3))

0.0958231000113301


In [8]:
import timeit
import numpy as np

Znp = np.random.randint(2, size=(1000,1000), dtype=np.int8)

def no_loop_int_view_reduce_rule(Z):
    N = np.zeros(Z.shape, dtype=np.int8)
    N[1:-1,1:-1] += (Z[ :-2, :-2] + Z[ :-2,1:-1] + Z[ :-2,2:] +
                     Z[1:-1, :-2]                + Z[1:-1,2:] +
                     Z[2:  , :-2] + Z[2:  ,1:-1] + Z[2:  ,2:])

    N_ = N.ravel()
    Z_ = Z.ravel()

    K = np.argwhere((N_== 2) & (Z_ == 1))
    Z_[...] = 0
    Z_[K] = 1
    Z_[np.argwhere(N_== 3)] = 1 # 规则化简，只计算一个mask数组

print(timeit.timeit(lambda: no_loop_int_view_reduce_rule(Znp), number=3))

0.058516499993856996


In [9]:
import timeit
import numpy as np

Znp = np.random.randint(2, size=(1000,1000), dtype=np.int8)

# 只有索引表达式的解
def no_loop_int_view_reduce_rule_no_mask(Z):
    N = np.zeros(Z.shape, dtype=np.int8)
    N[1:-1,1:-1] += (Z[ :-2, :-2] + Z[ :-2,1:-1] + Z[ :-2,2:] +
                     Z[1:-1, :-2]                + Z[1:-1,2:] +
                     Z[2:  , :-2] + Z[2:  ,1:-1] + Z[2:  ,2:])


    Z[...] = 0
    Z[1:-1,1:-1][(N == 3)[1:-1,1:-1]] = 1                 
    
print(timeit.timeit(lambda: no_loop_int_view_reduce_rule_no_mask(Znp), number=3))

0.04360699997050688


### 四行解决问题，除去两行置零操作，只有两行！

### 但这个解有错！因为改变了繁衍规则，等于放开了“二胎”。

In [11]:
import timeit
import numpy as np

Znp = np.random.randint(2, size=(1000,1000), dtype=np.int8)

# 只有索引表达式的解
def no_loop_int_view_reduce_rule_no_mask(Z):
    N = np.zeros(Z.shape, dtype=np.int8)
    N[1:-1,1:-1] += (Z[ :-2, :-2] + Z[ :-2,1:-1] + Z[ :-2,2:] +
                     Z[1:-1, :-2]                + Z[1:-1,2:] +
                     Z[2:  , :-2] + Z[2:  ,1:-1] + Z[2:  ,2:])

    K2 = (((N == 2)[1:-1,1:-1]) & (Z[1:-1,1:-1] == 1)) # 保留2
    L3 = (N == 3)[1:-1,1:-1]                           # 保留3，出生3
    Z[...] = 0
    Z[1:-1,1:-1][K2|L3] = 1                 
    
print(timeit.timeit(lambda: no_loop_int_view_reduce_rule_no_mask(Znp), number=3))

0.05661190004320815


### 让我们牺牲一点可读性
### 把生存法则编码成一个布尔表达式
### 省掉一个布尔矩阵创建

In [13]:
import timeit
import numpy as np

Znp = np.random.randint(2, size=(1000,1000), dtype=np.int8)

def no_loop_int_view_reduce_rule_no_mask(Z):
    N = np.zeros(Z.shape, dtype=np.int8)
    N[1:-1,1:-1] += (Z[ :-2, :-2] + Z[ :-2,1:-1] + Z[ :-2,2:] +
                     Z[1:-1, :-2]                + Z[1:-1,2:] +
                     Z[2:  , :-2] + Z[2:  ,1:-1] + Z[2:  ,2:])

    Z[...] = 0
    Z[1:-1,1:-1][(((N == 2)[1:-1,1:-1])&(Z[1:-1,1:-1] == 1))|(N == 3)[1:-1,1:-1]] = 1
    
print(timeit.timeit(lambda: no_loop_int_view_reduce_rule_no_mask(Znp), number=3))

0.056417200015857816


# 一行解
![gof apl](http://bazhou.blob.core.windows.net/learning/mpp/gof_apl.png)

# 深入讨论

- [用向量思考=改变描述问题的语言=用简化语言描述解决方案](http://bazhou.blob.core.windows.net/learning/mpp/467_Conway_s_Game_Of_Life_in_APL-a9xAKttWgP4.mp4)
- [APL implement](https://www.dyalog.com/download-zone.htm)
- [Alan Kay on APL](https://www.quora.com/What-made-APL-programming-so-revolutionary)
- [Notation as a Tool of Thought](https://bazhou.blob.core.windows.net/learning/mpp/iverson.pdf)

# 深入讨论

- [动画](http://bazhou.blob.core.windows.net/learning/mpp/game-of-life.mp4)显示进化过程自娱自乐


# 深入讨论

- 画出图形显示随着世界规模的增长，不同存储方案的显著性能差异
